## Pythia 70M Deduped Perplexity Analysis

This analysis was based off all the memorized and forgotten training sequences by the Pythia 70M dedupplicated model.

In [1]:
import csv
import torch
import numpy as np
import pandas as pd
import plotly.express as px
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, ReadInstruction
from torch.nn.functional import softmax
from tqdm import tqdm
tqdm.pandas()


/home/kyle/miniconda3/envs/memorization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m-deduped")
memories_tokens = load_dataset("EleutherAI/pythia-memorized-evals", split="deduped.70m").to_pandas()[["index", "tokens"]]
memories_ppls = pd.read_csv("70m_4_27/memories_deduped_70m.csv")
display(memories_tokens.head())
display(memories_ppls.head())
all_memories_data = memories_ppls.join(memories_tokens.set_index("index"), on="index", how="inner")
all_memories_data["text"] = all_memories_data["tokens"].progress_apply(lambda x: tokenizer.decode(x))
all_memories_data

Found cached dataset parquet (/home/kyle/.cache/huggingface/datasets/EleutherAI___parquet/EleutherAI--pythia-memorized-evals-623aaa371a33821a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


,index,tokens
0,441,"[5584, 4196, 1228, 187, 1036, 4, 209, 21723, 2..."
1,447,"[50262, 61, 2099, 92, 8861, 94, 187, 50262, 61..."
2,792,"[475, 50272, 953, 24781, 778, 320, 908, 281, 1..."
3,1539,"[424, 380, 16101, 313, 433, 17889, 3104, 10, 2..."
4,1705,"[3498, 2262, 2369, 40, 736, 13, 3956, 27, 21, ..."


,index,perplexity
0,441,2.935547
1,447,1.734375
2,792,3.556641
3,1539,3.658203
4,1705,9.070312


100%|██████████| 411448/411448 [00:10<00:00, 39889.20it/s]


,index,perplexity,tokens,text
0,441,2.935547,"[5584, 4196, 1228, 187, 1036, 4, 209, 21723, 2...",/(-96))\n16# *********************************...
1,447,1.734375,"[50262, 61, 2099, 92, 8861, 94, 187, 50262, 61...",\usepackage{upgreek}\n ...
2,792,3.556641,"[475, 50272, 953, 24781, 778, 320, 908, 281, 1...",* its contributors may be used to endors...
3,1539,3.658203,"[424, 380, 16101, 313, 433, 17889, 3104, 10, 2...",** The analytical (red dashed lines) and simul...
4,1705,9.070312,"[3498, 2262, 2369, 40, 736, 13, 3956, 27, 21, ...","px):noGrow,top:4dlu:noGrow,center:max(d;4px):n..."
...,...,...,...,...
411443,146431199,1.394531,"[281, 320, 669, 8604, 60, 805, 431, 1019, 8402...",to be $\documentclass[12pt]{minimal}\n ...
411444,146431278,2.791016,"[588, 1705, 285, 8415, 634, 1895, 15, 30952, 3...",will come and solve your problem.<!DOCTYPE ht...
411445,146431294,2.582031,"[15468, 13, 50275, 13743, 13, 50275, 15220, 13...","134, 135, 136, 137, 138, 139, 140,..."
411446,146431588,3.103516,"[27, 330, 14788, 10334, 14, 3429, 27, 577, 28,...",: C++; tab-width: 4; indent-tabs-mode: nil; c-...


In [4]:
# pile_path = "EleutherAI/pile-deduped-pythia-random-sampled"
# lower_index = 0
# upper_index = 50
# pile_tokens = load_dataset(pile_path, split=ReadInstruction("train", from_=lower_index, to=upper_index, unit="%", rounding="pct1_dropremainder")).to_pandas()
# pile_tokens
pile_tokens = pd.read_csv("70m_4_27/enriched_pile-1_deduped_70m.csv")
pile_tokens

/tmp/ipykernel_1714004/62905268.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pile_tokens = pd.read_csv("70m_4_27/enriched_pile-1_deduped_70m.csv")


,index,perplexity,tokens,text
0,18,50.406250,[ 15 46525 3439 ... 14 19 18647],.7375466\n-0.166555*29.7\n-4.9466835\nProduct ...
1,43,58.343750,[ 273 22523 18595 ... 281 2342 253],of ballot proposals in other states provide a...
2,86,23.953125,[ 749 10580 273 ... 3202 187 50274],subgraph of $K_n$ which is not isomorphic wit...
3,110,90.375000,[12556 187 71 ... 673 273 776],"tent\nfell and overturned, and the tent lay u..."
4,112,64.812500,[ 3847 277 2631 ... 10709 39021 15],"zy droid."" ""This is not okay."" ""Now this is po..."
...,...,...,...,...
2500229,73213617,108.562500,[1707 310 271 ... 1628 3074 619],There is an infinitesimal chance those projec...
2500230,73213631,51.187500,[13154 313 7779 ... 4171 2166 407],fibers (water fibers) of the frog glossophary...
2500231,73213695,74.918848,[ 1019 30077 14236 ... 2383 24 62],]{式（}{）} %定义公式编号样式\n\usetagform{newtag}\n\n\n%...
2500232,73213791,28.437500,[1798 2143 273 ... 281 253 7345],particular school of medicine about which he ...


In [ ]:
all_memories_data["memorized"] = True
pile_tokens["memorized"] = False
all_combined_data = pd.concat([all_memories_data, pile_tokens])
all_combined_data.to_csv("70m_4_27/combined_data_with_text.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
all_combined_data

,index,perplexity,tokens,text,memorized
0,441,2.935547,"[5584, 4196, 1228, 187, 1036, 4, 209, 21723, 2...",/(-96))\n16# *********************************...,True
1,447,1.734375,"[50262, 61, 2099, 92, 8861, 94, 187, 50262, 61...",\usepackage{upgreek}\n ...,True
2,792,3.556641,"[475, 50272, 953, 24781, 778, 320, 908, 281, 1...",* its contributors may be used to endors...,True
3,1539,3.658203,"[424, 380, 16101, 313, 433, 17889, 3104, 10, 2...",** The analytical (red dashed lines) and simul...,True
4,1705,9.070312,"[3498, 2262, 2369, 40, 736, 13, 3956, 27, 21, ...","px):noGrow,top:4dlu:noGrow,center:max(d;4px):n...",True
...,...,...,...,...,...
2500229,73213617,108.562500,[1707 310 271 ... 1628 3074 619],There is an infinitesimal chance those projec...,False
2500230,73213631,51.187500,[13154 313 7779 ... 4171 2166 407],fibers (water fibers) of the frog glossophary...,False
2500231,73213695,74.918848,[ 1019 30077 14236 ... 2383 24 62],]{式（}{）} %定义公式编号样式\n\usetagform{newtag}\n\n\n%...,False
2500232,73213791,28.437500,[1798 2143 273 ... 281 253 7345],particular school of medicine about which he ...,False


In [ ]:
pile_ppls = pd.read_csv("70m_4_27/pile-1_deduped_70m.csv")
display(pile_ppls)
# all_pile_data = pile_ppls.join(pile_tokens.set_index("index"), on="index", how="inner")
# all_pile_data = all_pile_data.drop(columns=["is_memorized"])
# all_pile_data["text"] = ""
# # all_pile_data["text"] = all_pile_data["tokens"].progress_apply(lambda x: tokenizer.decode(x))
# all_pile_data

## Analysis
- Distribution of perplexities
- Examples of how and low peprlexity sequences
- Histogram of the two perplexity distirbutions
- Code classifier

In [ ]:
memories_ppls["memorized"] = True
pile_ppls["memorized"] = False
combined_ppls = pd.concat([memories_ppls, pile_ppls])
combined_ppls

### Statistics

In [ ]:
combined_ppls = combined_ppls[combined_ppls["perplexity"] != np.inf]

print("Statics for memorized sequences")
display(pd.DataFrame(combined_ppls[combined_ppls["memorized"] == True]["perplexity"].describe()))

print("Statics for Pile (forgotten) sequences")
display(pd.DataFrame(combined_ppls[combined_ppls["memorized"] == False]["perplexity"].describe()))

In [ ]:
pple_sample = combined_ppls.sample(10000) 
pple_sample.columns

In [ ]:
fig = px.scatter(pple_sample, x="index", y="perplexity", color="memorized")
fig.update_layout(title_text="(Linear) Perplexity of 70m Deduped Pile vs. Memorized Sequences")
fig.show()

In [ ]:
np.log(pple_sample["perplexity"])

In [ ]:
fig = px.histogram(pple_sample, x=np.log(pple_sample["perplexity"]), color="memorized", log_y=True)
fig.update_layout(title_text="(Log-Log) Perplexity of 70m Deduped Pile vs. Memorized Sequences")
fig.show()

In [ ]:
fig = px.histogram(pple_sample, x=np.log(pple_sample["perplexity"]), color="memorized", log_y=False)
fig.update_layout(title_text="(Linear-Log) Perplexity of 70m Deduped Pile vs. Memorized Sequences")
fig.show()

### Memorized Examples

In [ ]:
all_memories_data["perplexity"].describe()["75%"]

In [ ]:
device = torch.device("cuda")
code_classifier = AutoModelForSequenceClassification.from_pretrained("usvsnsp/code-vs-nl").to(device).eval()
code_tokenizer = AutoTokenizer.from_pretrained("usvsnsp/code-vs-nl")

# all_memories_data_sample = all_memories_data.sample(100000)
# all_memories_data_sample
def is_code(text):
    tokens = code_tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True).to(device)
    outputs = code_classifier(**tokens)
    probabilities = softmax(outputs.logits.detach())
    return (probabilities[:, 0] > 0.457414).item()

all_memories_data_sample["is_code"] = all_memories_data_sample["text"].progress_apply(lambda x: is_code(x))

In [ ]:
all_memories_data_sample
display(all_memories_data_sample[all_memories_data_sample["perplexity"] > 1.10].value_counts("is_code"))
display(all_memories_data_sample[all_memories_data_sample["perplexity"] < 1.10].value_counts("is_code"))

In [ ]:
all_combined_data["is_code"] = all_combined_data["text"].progress_apply(lambda x: is_code(x))
all_combined_data